In [1]:
import mm_predictor # stat-analysis library
from sklearn import cross_validation, linear_model
import csv
import random
import numpy
import pandas as pd
#from xgboost import XGBClassifier

/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# intialize stat & elo dictionaries
mm_predictor.init()

In [3]:
# Load data
season_data = pd.read_csv('./ncaa-data/RegularSeasonDetailedResults.csv')
tourney_data = pd.read_csv('./ncaa-data/NCAATourneyDetailedResults.csv')
tourney_data = tourney_data[tourney_data.Season != 2017]

In [4]:
aggregated_data = pd.concat([season_data, tourney_data])

X,Y = mm_predictor.analyze_teams(aggregated_data)
# Run script

In [5]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import RobustScaler
from sklearn.svm import LinearSVC
from tpot.builtins import StackingEstimator
from xgboost import XGBClassifier

# NOTE: Make sure that the class is labeled 'target' in the data file
# tpot_data = pd.read_csv('PATH/TO/DATA/FILE', sep='COLUMN_SEPARATOR', dtype=np.float64)
# features = tpot_data.drop('target', axis=1).values
# training_features, testing_features, training_target, testing_target = \
#             train_test_split(features, tpot_data['target'].values, random_state=42)

# Score on the training set was:-0.5443811951009836
exported_pipeline = make_pipeline(
    StackingEstimator(estimator=XGBClassifier(learning_rate=0.01, max_depth=2, min_child_weight=4, n_estimators=100, nthread=1, subsample=0.15000000000000002)),
    RobustScaler(),
    StackingEstimator(estimator=GaussianNB()),
    StackingEstimator(estimator=LinearSVC(C=15.0, dual=True, loss="hinge", penalty="l2", tol=0.0001)),
    StackingEstimator(estimator=GaussianNB()),
    LogisticRegression(C=5.0, dual=False, penalty="l1")
)

exported_pipeline.fit(np.array(X), np.array(Y))


Pipeline(memory=None,
     steps=[('stackingestimator-1', StackingEstimator(estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.01, max_delta_step=0, max_depth=2,
       min_child_weight=4, missing=None, n_estimators=100, nthread=1,
       objective='binary:logistic...ty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [16]:
# perform simluation of 2017 NCAA tourney
print("Simulating 2017 NCAA tourney")
submission_data = []
tourney_seeds = pd.read_csv('./ncaa-data/TourneySeeds.csv')
tourney_2017_teams = tourney_seeds[tourney_seeds['Season'] == 2017]['Team']

stat_fields = ['score', 'fga', 'fgp', 'fga3', '3pp', 'ftp', 'or', 'dr',
                   'ast', 'to', 'stl', 'blk', 'pf']
print("Predicting matchups.")
tourney_2017_teams.sort_values()

for team_1 in tourney_2017_teams:
    for team_2 in tourney_2017_teams:
        if team_1 < team_2:
            prediction = mm_predictor.predict_winner(
                team_1, team_2, exported_pipeline, 2017, stat_fields)
            label = '2017_' + str(team_1) + '_' + str(team_2)
            submission_data.append([label, prediction[0][0]])


            

Simulating 2017 NCAA tourney
Predicting matchups.


[['2017_1437_1438', 0.56032965723366934],
 ['2017_1437_1458', 0.54589846723059288],
 ['2017_1437_1439', 0.57554777939428514],
 ['2017_1437_1452', 0.49928113667892315],
 ['2017_1437_1462', 0.67961892087474762],
 ['2017_1437_1455', 0.40068247710648741],
 ['2017_1437_1448', 0.65277047994774673],
 ['2017_1437_1457', 0.5410781316277985],
 ['2017_1181_1437', 0.27593370263000583],
 ['2017_1181_1196', 0.44345268844221775],
 ['2017_1181_1438', 0.47383695215291188],
 ['2017_1181_1374', 0.28402263174894893],
 ['2017_1181_1376', 0.58069568736531618],
 ['2017_1181_1458', 0.45959159481658374],
 ['2017_1181_1439', 0.48886708666642631],
 ['2017_1181_1266', 0.57089631255147233],
 ['2017_1181_1344', 0.52427616783583864],
 ['2017_1181_1425', 0.5259090403122364],
 ['2017_1181_1423', 0.39067633721637696],
 ['2017_1181_1190', 0.45016086491051055],
 ['2017_1181_1308', 0.46116885541118846],
 ['2017_1181_1407', 0.57442046105707001],
 ['2017_1181_1291', 0.67556946784313332],
 ['2017_1181_1309', 0.65974021953005

In [15]:
print("Writing %d results." % len(submission_data))
with open('./ncaa-data/submission.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['id', 'pred'])
    writer.writerows(submission_data)

Writing 2278 results.
